In [2]:
import pandas as pd


In [23]:
yearly_totals = pd.read_csv('/home/max/Documents/papers/regional-impacts-map/figure_data/yearly_totals.csv')
yearly_totals = yearly_totals[(yearly_totals.PY>1985) & (yearly_totals.PY<2020)].sort_values('PY').reset_index()
yearly_totals['label'] = yearly_totals.apply(
    lambda x: f'{x["PY"]:.0f} - {x["mid"]:,.0f} ({x["lb"]:,.0f}-{x["ub"]:,.0f})', axis=1
)
yearly_totals.to_json('data/yearly_totals.json',orient="records")
yearly_totals.tail()

,index,PY,lb,ub,mid,label
29,31,2015.0,4374.0,11061,6901.0,"2015 - 6,901 (4,374-11,061)"
30,32,2016.0,4714.0,12169,7559.0,"2016 - 7,559 (4,714-12,169)"
31,33,2017.0,5158.0,12848,8040.0,"2017 - 8,040 (5,158-12,848)"
32,34,2018.0,5473.0,13846,8567.0,"2018 - 8,567 (5,473-13,846)"
33,35,2019.0,6004.0,15078,9359.0,"2019 - 9,359 (6,004-15,078)"


In [35]:
impact_totals = pd.read_csv('/home/max/Documents/papers/regional-impacts-map/figure_data/impact_totals.csv')
impact_totals['label'] = impact_totals.apply(
    lambda x: f'{x["mid"]:,.0f} ({x["lb"]:,.0f}-{x["ub"]:,.0f})', axis=1
)
impact_totals['dataseries'] = "impacts"
#impact_totals['name'] = impact_totals.name.str.replace('and','and')
impact_totals.to_json('data/impact_totals.json',orient="records")
impact_totals.tail()

,name,mid,ub,lb,label,dataseries
1,Human and\nmanaged systems,12180,26130,6518,"12,180 (6,518-26,130)",impacts
2,"Mountains, snow and ice",6306,12225,3526,"6,306 (3,526-12,225)",impacts
3,"Rivers, lakes\n and soil moisture",12801,25152,7053,"12,801 (7,053-25,152)",impacts
4,Terrestrial ES,34974,65631,18516,"34,974 (18,516-65,631)",impacts
5,Other,23596,55682,9555,"23,596 (9,555-55,682)",impacts


In [34]:
continent_totals = pd.read_csv('/home/max/Documents/papers/regional-impacts-map/figure_data/continent_totals.csv')
continent_totals['label'] = continent_totals.apply(
    lambda x: f'{x["mid"]:,.0f} ({x["lb"]:,.0f}-{x["ub"]:,.0f})', axis=1
)
continent_totals['dataseries'] = "continents"
#impact_totals['name'] = impact_totals.name.str.replace('and','and')
continent_totals.to_json('data/continent_totals.json',orient="records")
continent_totals.tail()

,name,mid,ub,lb,label,dataseries
1,South America,4734,7754,3005,"4,734 (3,005-7,754)",continents
2,Europe,17746,28687,11191,"17,746 (11,191-28,687)",continents
3,Africa,7573,12205,4610,"7,573 (4,610-12,205)",continents
4,Asia,27638,41313,18700,"27,638 (18,700-41,313)",continents
5,Oceania,7404,11640,4626,"7,404 (4,626-11,640)",continents


In [61]:
df = pd.read_csv('/home/max/Documents/papers/regional-impacts-map/data/2_merged_da_data.csv')
df = df.sort_values(['LAT','LON'],ascending=[False, True])
df.head()

,LAT,LON,LAT_25,LON_25,LAT_5,LON_5,knutson_precip,updated_precip,knutson_temp,updated_temp,...,precip_da_cat,temp_da_cat,temp_da_binary,precip_da_binary,n_da_impacts,max_da_cat,study_addition,total_study_prop,area,is_land
10224,88.75,-178.75,88.75,-178.75,87.5,-177.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10225,88.75,-176.25,88.75,-176.25,87.5,-177.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10226,88.75,-173.75,88.75,-173.75,87.5,-172.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10227,88.75,-171.25,88.75,-171.25,87.5,-172.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False
10228,88.75,-168.75,88.75,-168.75,87.5,-167.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.327727,1685.654015,False


In [62]:
df.columns

Index(['LAT', 'LON', 'LAT_25', 'LON_25', 'LAT_5', 'LON_5', 'knutson_precip',
       'updated_precip', 'knutson_temp', 'updated_temp', 'population',
       'diff_temp', 'diff_precip', 'index', 'all_study_prop',
       'all_study_prop_upper', 'all_study_prop_lower', 'WA', 'temp_study_prop',
       'temp_study_prop_upper', 'temp_study_prop_lower', 'precip_study_prop',
       'precip_study_prop_upper', 'precip_study_prop_lower', 'precip_da_cat',
       'temp_da_cat', 'temp_da_binary', 'precip_da_binary', 'n_da_impacts',
       'max_da_cat', 'study_addition', 'total_study_prop', 'area', 'is_land'],
      dtype='object')

In [63]:
len(df.LAT.unique()),len(df.LON.unique())

(72, 144)

In [64]:
df = df.rename(columns={
    "updated_temp":"Temperature attribution category",
    "updated_precip":"Precipitation attribution category",
    "temp_study_prop": "Weighted studies on temperature",
    "all_study_prop": "Weighted studies"
})
d = {
    "type": "FeatureCollection",
    "features": []
}
df.loc[pd.isna(df['precip_da_cat']),"precip_da_cat"] = 5
df.loc[pd.isna(df['temp_da_cat']),"temp_da_cat"] = 5
cols = [
    "Temperature attribution category","Precipitation attribution category",
    "LAT","LON"
]
#cols = df.columns

for i, row in df.fillna(5).iterrows():
    rdict = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [row.LON-1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT-1.25],                
            ]
        }, 
        "properties": {
            
        }
    }
    rdict["Temperature"] = row["Temperature attribution category"]
    rdict["Precipitation"] = row["Precipitation attribution category"]
    for c in cols:
        if "LATasdf" in c or "LONasdf" in c:
            continue
        rdict[c] = row[c]
        rdict["properties"][c] = row[c]
    
    d['features'].append(rdict)
    
with open('data/da_data.json', 'w') as fp:
    json.dump(d, fp)

In [59]:
c = ["#ffed6f","#b2182b","#ef8a62","#fddbc7","#f7f7f7","#d1e5f0","#67a9cf","#2166ac","#33a02c"]

In [60]:
long_df = pd.read_csv('/home/max/Documents/papers/regional-impacts-map/data/2b_merged_da_data_impacts.csv')
long_df = long_df.sort_values(['LAT','LON'],ascending=[False, True])
#long_df['impact'] = [x.split()[0] for x in long_df['impact']]
long_df.head()

,index,LAT,LON,weight,driver,impact
10224,10224,88.75,-178.75,0.462597,all,all
20592,10224,88.75,-178.75,0.075289,all,Coastal and marine Ecosystems
30960,10224,88.75,-178.75,0.001570,all,Human and managed
41328,10224,88.75,-178.75,0.224276,all,"Mountains, snow and ice"
51696,10224,88.75,-178.75,0.015664,all,"Rivers, lakes, and soil moisture"


In [27]:
# df = df.rename(columns={
#     "updated_temp":"Temperature",
#     "updated_precip":"Precipitation",
#     "temp_study_prop": "Weighted studies on temperature",
#     "all_study_prop": "Weighted studies"
# })
d = {
    "type": "FeatureCollection",
    "features": []
}
cols = [x for x in long_df.columns if x!="index"]
for i, row in long_df.fillna(5).iterrows():
    rdict = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [row.LON-1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT+1.25],
                [row.LON+1.25,row.LAT-1.25],
                [row.LON-1.25,row.LAT-1.25],                
            ]
        }, 
        "properties": {
            
        }
    }
    for c in cols:
        if "LATasdf" in c or "LONasdf" in c:
            continue
        rdict[c] = row[c]
        rdict["properties"][c] = row[c]
    
    d['features'].append(rdict)
    
with open('data/ws_data.json', 'w') as fp:
    json.dump(d, fp)